In [ ]:
%cd /content
!git clone https://github.com/rica-team/rica-server.git
%cd /content/rica-server

!pip install .[pt]

In [ ]:
from huggingface_hub import login

login()

In [ ]:
import json
from rica.server import RiCA
from rica.connector import transformers_adapter as tf

In [ ]:
# 1. Create the RiCA application instance first.
app = RiCA()

In [ ]:
# 2. Register tools with the application instance.
@app.register("sys.python.exec", background=False, timeout=5000)
async def _sys_python_exec(input_):
    """
    A tool to execute a single line of Python code.
    input:
    {
        "code": "1+1"
    }
    output:
    {
        "result": "2"
    }
    """
    try:
        code = input_.get("code", "")
        # Using eval is unsafe. For a real application, use a safer execution environment.
        result = eval(code)
        return {"result": str(result)}
    except Exception as e:
        return {"error": str(e)}

In [ ]:
# 3. Create the ReasoningThread, passing the app instance to it.
rt = tf.ReasoningThread(app, model_name="google/gemma-3-1b-it")

In [ ]:
# 4. Register callbacks.
@rt.token_generated
def _on_token(token: str):
    # This function receives every single token generated by the model.
    # It represents the model's "thinking" process, including tool calls.
    print(token, end="", flush=True)


@rt.trigger
def _on_response(response_payload: any):
    # This function is ONLY called when the model uses the `rica.response` tool.
    # It's for showing the final, user-facing answer.
    print("\n\n--- USER-FACING RESPONSE ---")
    # Assuming the response payload is a list of content blocks, as per the prompt.
    if isinstance(response_payload, list):
        for item in response_payload:
            if item.get("type") == "text":
                print(item.get("content", ""))
    else:
        print(json.dumps(response_payload, indent=2))
    print("----------------------------\n")

In [ ]:
async def main():
    print("--- Starting RiCA Demo ---")

    # 5. Start the reasoning thread. It will initialize the model and wait for input.
    # The `run()` call here is important to start the loop, even before first insert.
    rt.run()
    # Initially, it will pause itself waiting for input.

    # 6. Insert the initial user prompt. The adapter will format it correctly.
    await rt.insert(
        "Please calculate 123*456 using sys.python.exec. "
        "Think step-by-step about what you need to do, and then use the "
        "rica.response tool to give me the final answer."
    )

    # 7. Wait for the generation to complete (e.g., hit EOS or pause after tool call).
    print(rt.context)
    await rt.wait()

    print("\n\n--- Final Context ---")
    print(rt.context)
    print("---------------------")

    # 8. Clean up resources.
    await rt.destroy()

In [ ]:
await main()